In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import difflib

In [ ]:
meta = pd.read_csv('data/litti_et_al/meta.csv')
meta.head()

In [ ]:
data = pd.read_csv('data/litti_et_al/combined_transform.csv', index_col=0)
data.shape

In [ ]:
data.shape, meta.shape

In [ ]:
metaStrains = meta.Strain.unique()
metaConditions = meta.Condition.unique()

meta['snpStrain'] = meta.Strain.str.extract('[0-9 ]*\([a-zA-z]*, ([a-zA-Z0-9-]*).*\)').replace(np.nan, '').astype(str)
metaStrainToSnp = meta.snpStrain.unique()

conditionGroup = meta.groupby('Condition')

In [ ]:
snp = pd.read_csv('data/litti_et_al/sgrp_Sc_SNPs.txt', sep='\t')
snp.head()

In [ ]:
snpStrains = snp['mutant strains.1'].str.strip().str.replace('|', '').str.split().values

snpStrainSet = set()
for strains in snpStrains:
    for s in strains:
        snpStrainSet.add(s)
snpStrainSet = list(snpStrainSet)
snpStrainSet

In [ ]:
dist = np.zeros((len(metaStrainToSnp), len(snpStrainSet)))

for i, m in enumerate(metaStrainToSnp):
    for j, s in enumerate(snpStrainSet):
        dist[i,j] = difflib.SequenceMatcher(a=m, b=s).ratio()

In [ ]:
strainMap = {}

for j, s in enumerate(snpStrainSet):
    i = np.where(dist[:,j] == dist[:,j].max())[0][0]
    strainMap[s] = metaStrainToSnp[i]

strainMap

In [ ]:
genotype = np.zeros((snp.shape[0], len(snpStrainSet)), dtype=int)

for i in range(snp.shape[0]):
    for s in snpStrains[i]:
        genotype[i, snpStrainSet.index(s)] = 1
        
genotype = pd.DataFrame(genotype, columns=[strainMap[s] for s in snpStrainSet])
genotype.head()

In [ ]:
cond = metaConditions[1]

for gi in range(100, 110):
    g = genotype.iloc[gi, :]

    ind = conditionGroup.get_group(cond)
    ind = pd.merge(ind, g.to_frame('genotype'), right_index=True, left_on='snpStrain')

    temp = data.iloc[:, ind.index]
    
    plt.figure()
    for i in range(temp.shape[1]):
        plt.subplot(1,2,ind.iloc[i,:]['genotype']+1)
        plt.plot(data.index, temp.iloc[:,i], c='C0')
        #plt.plot(data.index, temp.iloc[:,i], color='C%d'%ind.iloc[i,:]['genotype'])
        
    plt.subplot(121)
    plt.ylim(temp.min().min(), temp.max().max())
    plt.subplot(122)
    plt.ylim(temp.min().min(), temp.max().max())
    